<a href="https://colab.research.google.com/github/SonnyDev/LearnFrench/blob/master/New_TER_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install transformers library
!pip install transformers==3.1.0

#import the Transformers pipeline library
from transformers import pipeline

#Preprocessing and visualization libraries
import plotly.express as px
import pandas as pd
import numpy as np

In [ ]:
#Define the classifier with multi-lingual option
zsmlc_classifier = pipeline("zero-shot-classification", 
                            model='joeddav/xlm-roberta-large-xnli')


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
news_data = pd.read_csv('/content/drive/My Drive/bbc-text.csv')

In [36]:
#Single prediction's output
sequences = "The world's most valuable cryptocurrency was down 5% Monday after plunging again over the weekend. Bitcoin prices have now plummeted nearly 15% in the past week. At a price of just below $33,000, bitcoin is more than 50% below its record high of near $69,000 from late last year and at its lowest point of 2022. Other cryptocurrencies, sometimes referred to as altcoins, have been hit hard too. Ethereum, binance, solana and cardano are all down about 15% in the past week, while Elon Musk's beloved dogecoin has tumbled 10%. Cryptocurrencies are proving to be just as risky as stocks and susceptible to the same concerns that are dragging down the Dow, S&P 500 and Nasdaq.'Volatile trading in digital assets has not been that unusual in previous years', said Michael Kamerman, CEO of trading platform Skilling. 'Cryptocurrencies are increasingly moving in sync with tech stocks with investors treating both as risk assets and often retreating to safer corners of the market during bouts of market volatility.'"
#sequences = news_data.iloc[1]['text']

candidates_labels = list(news_data.category.unique())
#candidate_labels = ['science', 'politics', 'business', 'tech', 'entertainment']

result = zsmlc_classifier(sequences, candidates_labels, multi_class= True)

result

{'labels': ['tech', 'business', 'sport', 'entertainment', 'politics'],
 'scores': [0.817554771900177,
  0.6502110362052917,
  0.3513646721839905,
  0.324686199426651,
  0.30529525876045227],
 'sequence': "The world's most valuable cryptocurrency was down 5% Monday after plunging again over the weekend. Bitcoin prices have now plummeted nearly 15% in the past week. At a price of just below $33,000, bitcoin is more than 50% below its record high of near $69,000 from late last year and at its lowest point of 2022. Other cryptocurrencies, sometimes referred to as altcoins, have been hit hard too. Ethereum, binance, solana and cardano are all down about 15% in the past week, while Elon Musk's beloved dogecoin has tumbled 10%. Cryptocurrencies are proving to be just as risky as stocks and susceptible to the same concerns that are dragging down the Dow, S&P 500 and Nasdaq.'Volatile trading in digital assets has not been that unusual in previous years', said Michael Kamerman, CEO of trading pl

In [28]:
print(sequences)

President Joe Biden is vowing to ratchet up even more pressure on Vladimir Putin even as the Russian leader dug in and defended the war in Ukraine at a nationalistic pageant featuring thousands of troops in Moscow. Russia's Victory Day celebration on Monday followed a stunning sequence of revelations about the deadly results of US intelligence sharing with Ukraine and after Biden called for a staggering $33 billion haul of arms and aid for Kyiv in another extraordinary step that widened US involvement in the war.


In [37]:
#Delete the sequence key and create the final dataframe
result_df = pd.DataFrame(result)

# Plot the result
fig = px.bar(result_df, x='labels', y='scores')
fig.show()

In [38]:
#CLASSIFICATION OF ENGLISH TEXT
# Get subset of data
def select_subset_data(data, label_column, each_target_size = 2):

  all_targets = list(data[label_column].unique())
  list_dataframes = []

  for label in all_targets:
    subset = data[data[label_column]==str(label)]
    subset = subset.sample(each_target_size)

    list_dataframes.append(subset)

  return pd.concat(list_dataframes)


def make_prediction(clf_result):

  # Get the index of the maximum probability score
  max_index = np.argmax(clf_result["scores"])
  predicted_label = clf_result["labels"][max_index]

  return predicted_label


def run_batch_prediction(original_data, label_column, desc_col, my_classifier = zsmlc_classifier):

  # Make a copy of the data
  data_copy = original_data.copy()

  # The list that will contain the models predictions
  final_list_labels = []

  for index in range(len(original_data)):
    # Run classification
    sequences = original_data.iloc[index][desc_col]
    candidate_labels = list(original_data[label_column].unique())
    result = my_classifier(sequences, candidate_labels, multi_class = True)

    # Make prediction
    final_list_labels.append(make_prediction(result))
  
  # Create the new column for the predictions
  data_copy["clf_predictions"] = final_list_labels

  return data_copy

In [40]:
# Get the subset of dataframe
subset_news_data = select_subset_data(news_data, "category")

# Run the predictions on the new dataset
pred_res_data = run_batch_prediction(subset_news_data, "category", "text")
pred_res_data

,category,text,clf_predictions
1531,tech,security scares spark browser fix microsoft is...,tech
2103,tech,ds aims to touch gamers the mobile gaming indu...,entertainment
159,business,orange colour clash set for court a row over t...,tech
1104,business,unilever shake up as profit slips anglo-dutch ...,business
1305,sport,parry relishes anfield challenge bbc sport ref...,business
980,sport,claxton hunting first major medal british hurd...,sport
766,entertainment,applegate s charity show closes us musical swe...,entertainment
939,entertainment,dame julie pops in to see poppins mary poppins...,entertainment
1985,politics,ban on hunting comes into force fox hunting wi...,politics
1024,politics,labour attacked on howard poster labour has be...,business
